# Day 24 - Hex grid

* https://adventofcode.com/2020/day/24

Oh, goodie, a [hex grid](https://www.redblobgames.com/grids/hexagons/) problem! I enjoy those :-) We were given a hex grid problem before, on [Day 11 of AoC 2017](../2017/Day%2011.ipynb), and I've come across a handful of applications of such grids elsewhere since (e.g. the [Hexagon Beam Max Sum code kata](https://www.codewars.com/kata/5ecc1d68c6029000017d8aaf/)).

For part 1, we basically need 3 things:

- Code to split the instructions into distinct directions (`w`, `e`, `sw`, `se`, `nw` and `ne`)
- Our state: A dictionary tracking what tiles we visited and flipped, because we may need to flip them more than once, and our current hex-grid position.
- A way to map a direction to our next grid coordinate.

The latter is actually really easy; each of the 6 directions can be expressed as a pair of deltas (one each of `-1`, `0`, and `1`, see [neighbours using axial coordinates](https://www.redblobgames.com/grids/hexagons/#neighbors)).

In [1]:
from collections.abc import Iterable, Iterator, MutableMapping
from dataclasses import dataclass, field
from enum import Enum
from operator import attrgetter
from typing import Any

@dataclass(frozen=True)
class HexPos:
    q: int = 0
    r: int = 0

    def __add__(self, other: Any) -> "HexPos":
        if not isinstance(other, HexPos):
            return NotImplemented
        return type(self)(self.q + other.q, self.r + other.r)

class HexDir(Enum):
    w = HexPos(-1, 0)
    e = HexPos(1, 0)
    nw = HexPos(0, -1)
    ne = HexPos(1, -1)
    sw = HexPos(-1, 1)
    se = HexPos(0, 1)

    @classmethod
    def from_steps(cls, steps: str) -> Iterator["HexPos"]:
        chars = iter(steps)
        for step in chars:
            if step not in "ew":
                step += next(chars)
            yield cls[step].value

class Tile(Enum):
    white = 0
    black = 1

    def flip(self) -> "Tile":
        return Tile(1 - self.value)

@dataclass
class TileFloor:
    tiles: MutableMapping[HexPos] = field(default_factory=dict)

    @classmethod
    def from_instructions(cls, instructions: Iterable[str]) -> "TileFloor":
        floor = cls()
        tiles = floor.tiles
        for steps in instructions:
            pos = sum(HexDir.from_steps(steps), HexPos())
            tiles[pos] = tiles.get(pos, Tile.white).flip()
        return floor

    @property
    def black_count(self) -> int:
        return sum(map(attrgetter("value"), self.tiles.values()))


test_floor = TileFloor.from_instructions("""\
sesenwnenenewseeswwswswwnenewsewsw
neeenesenwnwwswnenewnwwsewnenwseswesw
seswneswswsenwwnwse
nwnwneseeswswnenewneswwnewseswneseene
swweswneswnenwsewnwneneseenw
eesenwseswswnenwswnwnwsewwnwsene
sewnenenenesenwsewnenwwwse
wenwwweseeeweswwwnwwe
wsweesenenewnwwnwsenewsenwwsesesenwne
neeswseenwwswnwswswnw
nenwswwsewswnenenewsenwsenwnesesenew
enewnwewneswsewnwswenweswnenwsenwsw
sweneswneswneneenwnewenewwneswswnese
swwesenesewenwneswnwwneseswwne
enesenwswwswneneswsenwnewswseenwsese
wnwnesenesenenwwnenwsewesewsesesew
nenewswnwewswnenesenwnesewesw
eneswnwswnwsenenwnwnwwseeswneewsenese
neswnwewnwnwseenwseesewsenwsweewe
wseweeenwnesenwwwswnew
""".splitlines())

assert test_floor.black_count == 10

In [2]:
import aocd
tile_instructions = aocd.get_data(day=24, year=2020).splitlines()

In [3]:
tile_floor = TileFloor.from_instructions(tile_instructions)
print("Part 1:", tile_floor.black_count)

Part 1: 400


## Part 2 - Hexagonal cellular automatons!

More fun! Using axial coordinates, we can trivially put our tiles into a numpy grid and use the standard `scipy.signal.convolve2d()` approach to progress each state.

Of course, we do then need a custom kernel; the old 3x3 with 8 out of 9 cells set to 1 won't cut it. But with a [basic 2d-array for storage](https://www.redblobgames.com/grids/hexagons/#map-storage) (and so, super simple to use a numpy array for this), the kernel to use is still really simple; still a 3x3 array, but with 6 cells set to 1, matching the [6 neighbours](https://www.redblobgames.com/grids/hexagons/#neighbors) I mentioned before:

```
0  1  1
1  0  1
1  1  0
```

We do need to remember to grow the matrix each round; the tile floor size is infinite.

In [4]:
import numpy as np
from scipy.signal import convolve2d

_kernel = np.array([[0, 1, 1], [1, 0, 1], [1, 1, 0]])

class ArtExhibitTileFloor:
    def __init__(self, tilefloor: TileFloor) -> None:
        minpos, maxpos = HexPos(), HexPos()
        for hpos in tilefloor.tiles:
            if hpos.q < minpos.q:
                minpos = HexPos(hpos.q, minpos.r)
            elif hpos.q > maxpos.q:
                maxpos = HexPos(hpos.q, maxpos.r)
            if hpos.r < minpos.r:
                minpos = HexPos(minpos.q, hpos.r)
            elif hpos.r > maxpos.r:
                maxpos = HexPos(maxpos.q, hpos.r)
        shape = (maxpos.r - minpos.r + 1, maxpos.q - minpos.q + 1)
        self._matrix = matrix = np.zeros(shape, dtype=np.uint8)
        for hpos, tile in tilefloor.tiles.items():
            matrix[hpos.r - minpos.r, hpos.q - minpos.q] = tile.value

    @property
    def black_count(self) -> int:
        return np.sum(self._matrix)

    def run(self, rounds: int = 100) -> int:
        f = self._matrix
        full = {t: np.full(f.shape, t.value) for t in Tile}
        for _ in range(rounds):
            f = np.pad(f, 1, constant_values=Tile.white.value)
            full = {t: np.pad(m, 1, constant_values=t.value) for t, m in full.items()}
            counts = {tile: convolve2d(f == tile.value, _kernel, mode='same') for tile in Tile}
            rules = {
                # Any **black** tile with **zero** or **more than 2** black tiles immediately
                # adjacent to it is flipped to **white**.
                Tile.white: (f == Tile.black.value) & ((counts[Tile.black] == 0) | (counts[Tile.black] > 2)),
                # Any **white** tile with **exactly 2** black tiles immediately adjacent to it
                # is flipped to **black**.
                Tile.black: (f == Tile.white.value) & (counts[Tile.black] == 2),
            }
            f = np.select(list(rules.values()), [full[t] for t in rules], default=f)
            self._matrix = f
        return self.black_count

assert ArtExhibitTileFloor(test_floor).run() == 2208

In [5]:
print("Part 2:", ArtExhibitTileFloor(tile_floor).run())

Part 2: 3768
